In [1]:
import pandas as pd
import numpy as np
import glob
from nba_api.stats.static.players import *

import matplotlib.pyplot as plt
import itertools
import seaborn as sns
sns.set_style('darkgrid')
from sklearn.linear_model import RidgeCV
from typing import Callable, List

# Plus Minus Player Impact Metrics

A common question in BBall analytics is how do we quantify the effect that a single player has on the outcome of a game.

The simplest metric to capture this is Plus-Minus (PM), which assigns to each player (per game) a value which quantifies how much the game score changed for their team when the player was on the court.  For example, if, in all of the minutes that Andre Drummond is on the court, the Lakers score 20 points but their opponents score 30 points, then Andre Drummond is assigned a PM of -10.

However, a major problem with this metric is that it does not consider the effects of lineups.  For example, if Andre Drummond only plays with Anthony Davis, does the -10 value really accurately capture the effect of Andre Drummond?  Or, if Andre Drummond is specifically played against a certain matchup, how does this change his contribution?

How can we correct for this?

## Regularized Adjusted Plus Minus

One attempt to account for these effects is the Regularized Adjusted Plus-Minus (RAPM) metric.  The RAPM metric directly models the fact that lineups matter when we consider the impact of a player.  However, in order to do this, we have a more complicated problem.

To start, instead of looking at the whole game for a player, we look at specific lineups within a single game.  Suppose that in a segment of the game with a fixed 10 man lineup (5 home, 5 away), the home team scores 20 points and the away team scores 30 points.  Then in this segment, commonly referred to as a stint, in a normal plus/minus system, we can say that all 5 home players earned -10 and all 5 away players earned +10.  

However, it's not the individual, but rather the lineup of ($h_1, h_2, h_3, h_4, h_5$) and ($a_1, a_2, a_3, a_4, a_5$) that collectively contributes to a score differential of -10.  A simple mathematical model for this segment can be:

$$
\begin{equation}
    -10 = \beta_{1} h_1 + \beta_{2} h_2 + \beta_{3} h_3 + \beta_{4} h_4 + \beta_{5} h_5 - \beta_{6} a_1 - \beta_{7} a_2 - \beta_{8} a_3 - \beta_{9} a_4 - \beta_{6} a_5
\end{equation}
$$

where $\beta$ quantifies how much player $h_1$ contributes to the collective $-10$.  Due to the inherent symmetry of the problem, where if we reversed home and away, the value would be $+10$, we assign the opposite sign for team $a$.  In practice, $h_1$ would instead be an actual player, like Andre Drummond, so every player in the league would have a unique $\beta$ that quantifies how much they contribute.  The estimated $\beta$ is the RAPM assigned to each player!  


### Estimating $\beta$

For every stint over every single season, we can get a value and form a similar equation as above, filling in the players that make up that lineup.  Under this statistical model, we can estimate $\beta$ by treating this problem as massive linear regression where every row takes the form of:

$$
\begin{equation}
    y_j = \sum_{i=0}^{N} \beta_{i} x_{i,j}  a_{i,j}
\end{equation}
$$

where there are $N$ unique players and $x_{i,j} \in \{0, 1\}$ is an binary variable where $x_{i,j} = 1$ if player $i$ takes place in the $j$-th stint and $a_{i,j} \in \{-1, 1\}$ is another binary variable where $a_{i,j} = -1$ if player $i$ is part of the away team in the $j$-th stint.

So to compute RAPM, we need to estiamte $\beta$ and thus we have the following steps:

1. Get every stint from play-by-play data
2. Build the linear regression problem
3. Estimate $\beta$ using whatever method

### 1. Getting Stints from PBP Data

For a given game, we can build a collection of stints by understanding that the boundaries of a stint are governed by period starts, period ends, and substitutions.  Only in these events do lineups every change.  Using this simple logic, we can build a stint as follows:

1. Start with a `current_lineup`
2. Accumulate all of the plays that happen between the start of the stint and either a substitution or the period ends, compute values $y_j$
3. Attribute $y_j$ to the `current_lineup`
4. Advance time to the time of the substitution and make changes to the `current_lineup` with `updated_current_lineup` based on the substitutions 
5. Start a new stint with `current_lineup = updated_current_lineup`, go back to 1

In [8]:
def get_stints_from_pbp(pbp : pd.DataFrame, 
                        starters : pd.DataFrame, 
                        summarize_stint : Callable[[pd.DataFrame, List[int]], List[float]]) -> List[float]:
    
    # get the two teams playing - formated as [away_team_abbrev, home_team_abbrev]
    first = pbp[(~pd.isnull(pbp['PLAYER2_TEAM_ABBREVIATION'])) & \
                (~pd.isnull(pbp['PLAYER1_TEAM_ABBREVIATION']))].head(1)
    teams = first[['PLAYER2_TEAM_ABBREVIATION','PLAYER1_TEAM_ABBREVIATION']].values.tolist()[0]
    stints = []
    # scores are represented as [away_team_points, home_team_score] to align with the presentation
    # in the dataset
    current_score = [0, 0]
    for period in [1,2,3,4]:
        
        # get the period starters and build the current lineup
        period_starters = starters[starters['PERIOD'] == period]
        away_team = set(period_starters[period_starters['TEAM_ABBREVIATION'] == teams[0]]['PLAYER_ID'].values)
        home_team = set(period_starters[period_starters['TEAM_ABBREVIATION'] == teams[1]]['PLAYER_ID'].values)
        
        current_time = 0
        substitutions = pbp[(pbp["EVENTMSGTYPE"] == 8) & (pbp['PERIOD'] == period)]
        for k, v in substitutions.groupby(["time"], sort = False):

            lineup = list(home_team) + list(away_team)
            if len(lineup) == 10:
                pbp_interval = pbp[(pbp['PERIOD'] == period) & 
                                   (pbp["time"] > current_time) & 
                                   (pbp["time"] <= k)]


                vals = summarize_stint(pbp_interval, current_score)
                # accumulate all of the plays within this stint
                if vals != np.inf:
                    stints.append([k - current_time] + lineup + vals)

                # keep track of the current score - this is necessary for some metrics
                # and because of the way the score is presented
                scores = pbp_interval[~pbp_interval['SCORE'].isna()]['SCORE']
                if scores.shape[0] >= 1:
                    current_score = map(int, scores.iloc[-1].split("-"))
                
            
            # advance to next time and make all player substitutions
            # at the final time
            current_time = k
            for _, r in v.iterrows():
                team = r['PLAYER1_TEAM_ABBREVIATION']
                if team == teams[0]:
                    t = away_team
                else:
                    t = home_team
                
                pid_out = r['PLAYER1_ID']
                pid_in = r['PLAYER2_ID']
                t.remove(pid_out)
                t.add(pid_in)
        
        # handle final period stint - this is skipped by our loop
        lineup = list(home_team) + list(away_team)
        if len(lineup) == 10:

            pbp_interval = pbp[(pbp['PERIOD'] == period) & 
                               (pbp["time"] > current_time)]

            vals = summarize_stint(pbp_interval, current_score)   
            if vals != np.inf:
                stints.append([720 - current_time] + lineup + vals)
            
            # current score carries over to next period
            scores = pbp_interval[~pbp_interval['SCORE'].isna()]['SCORE']
            if scores.shape[0] >= 1:
                current_score = map(int, scores.iloc[-1].split("-"))
            
    return stints

#### Different Metrics

The above function is generic in how we compute the $y$ in our model.  It takes in a function that takes in a `pd.DataFrame` and produces a set of values to summarize the stint.

The function we use for this is the P/M normalized to per 100 possessions.  This is approximated as:

$$
\begin{equation}
    y = \frac{100(\text{home_pts} - \text{away_pts})}{N_p}
\end{equation}
$$
where `home_pts` is the points scored by the home team, `away_pts` are the points scored by the away team, and $N_p$ is the total number of possessions in a given stint.  This essentially assumes that if we were to play the same lineup for 100 possessions, then the differential would end up being $y$.

In [9]:
def compute_score_diff_per_100(pbp : pd.DataFrame, current_score : List[int]) -> List[float]:
    if pbp.shape[0] <= 1:
        return np.inf
    else:
        scores = pbp[~pbp['SCORE'].isna()]['SCORE']
        if scores.shape[0] > 0:
            start_away, start_home = current_score
            end_away, end_home = map(int, scores.iloc[-1].split("-"))

            home_points = end_home - start_home
            away_points = end_away - start_away
        else:
            home_points = away_points = 0
        possesions = pbp[pbp["EVENTMSGTYPE"].isin([1,2,4,5])].shape[0]
        if possesions <= 2:
            return np.inf
        return [100 * (home_points - away_points)/possesions, possesions]

### Running the Numbers

Now that we have the code, we interate over every game in a season and get all of the stints

In [10]:
def get_all_stints():
    seasons = ['2013-14', '2014-15', '2015-16', '2016-17', '2017-18', '2018-19', '2019-20']
    season_dfs = []
    for season in seasons:
        files = [f for f in glob.glob("../../data/pbp/{}/*.csv".format(season)) if "_starters" not in f]
        all_stints = []
        for f in files:
            try:
                pbp = pd.read_csv(f)
                pbp['SCOREMARGIN'].replace('TIE', 0, inplace = True)
                pre = f.split(".csv")[0]
                starters = pd.read_csv(pre + "_starters.csv")
                stints = get_stints_from_pbp(pbp, starters, compute_score_diff_per_100)
                all_stints += stints
            except KeyError:
                # Occasional errors in the data lead to throwing out the stints for a game
                pass
        season_dfs.append(pd.DataFrame(all_stints, columns = ['duration (s)'] + ['h{}'.format(i + 1) for i in range(5)] + ['a{}'.format(i + 1) for i in range(5)] + ['diff', 'poss']))
    return season_dfs

In [11]:
season_dfs = get_all_stints()

To get an idea of what the processed data for a stint looks like:

In [12]:
season_dfs[0].head()

,duration (s),h1,h2,h3,h4,h5,a1,a2,a3,a4,a5,diff,poss
0,389,200768,101161,203082,201942,202685,202689,203106,201945,2744,201177,-7.500000,40
1,103,200768,101161,202349,201942,202335,202689,201945,201196,203469,2744,0.000000,9
2,21,200768,101161,202349,201942,202335,202689,201945,201196,203469,202687,25.000000,4
3,53,200768,202349,201942,201946,202335,202689,201945,201196,203469,202687,-25.000000,8
4,154,200768,202349,2422,201946,202335,202689,203106,201196,203469,202687,4.761905,21


### Estimating the $\beta$

Before we can fully estimate the $\beta$ RAPM coefficients, we need to assign each player a one-hot encoding.  Then, as the above code returns a stint as a list of 10 players that were in the game, we need to translate those 10 players according to our one-hot encoding.  Once this is done, we have a pretty standard linear regression problem.

As there is a high-level of ambiguity in properly estimating the coefficients, especially since we care about the interpretation of the values, typical estimation is done using a regularized method such as Ridge-Regression (l2-reg).  This method encourages values of $\beta$ to be "small" with the idea that no $\beta$ should be substantially higher than others.  

Other regularization methods could be used, however assumptions need to be checked.  For example, the LASSO are not appropriate as we do not expect a players contributions to be identically 0, which is what the LASSO aims to do.

In [24]:
def compute_RAPM_table(all_stints):
    # Get the list of all players and their ids using nba_api
    players = get_players()
    
    # Convenience dictionaries mapping id to name and back
    player_dict = {p['id'] : p['full_name'] for p in players}
    inverse_player_dict = {p['full_name'] : p['id'] for p in players}
    
    # Get the list of all unique players in the stints
    all_players = [all_stints[h].unique().tolist() for h in ['h{}'.format(i + 1) for i in range(5)] + ['a{}'.format(i + 1) for i in range(5)]]
    all_players = sorted(list(set(list(itertools.chain.from_iterable(all_players)))))
    
    # Create one-hot encodings per-player
    encodings = pd.get_dummies(all_players).astype(float)
    
    X = []
    y = []
    weights = []
    
    # Turn stints into one-hot representation (assigning + to home teams)
    # Also get the value to regress on
    for x, stint in all_stints.iterrows():
        res = None
        for id in stint[['h{}'.format(i + 1) for i in range(5)]].values:
            if res is None:
                res = encodings[id].values.copy()
            else:
                res += encodings[id].values.copy()

        for id in stint[['a{}'.format(i + 1) for i in range(5)]].values:
            res -= encodings[id].values.copy()

        X.append(res)
        y.append(stint['diff'])
        weights.append(stint['poss'])
    X = np.vstack(X)
    y = np.vstack(y)[:,0]
    
    # Estimate the betas using ridge-regression, standard method
    weights = np.vstack(weights)
    model = RidgeCV(alphas=(np.array([0.01,0.1,1.0,10,100,500,1000,2000,5000])),cv=5)
    model.fit(X, y, sample_weight = all_stints['poss'].values)
    
    # Create a dataframe from the estimated coefficients with the player names and number of stints
    # attached
    X_df = pd.DataFrame(X, columns=encodings.columns)
    coeff_mappings = list(zip(encodings.columns, model.coef_))
    coeff_mappings = sorted(coeff_mappings, key = lambda x : -x[1])
    rapm_estimates = []
    for ii, c in coeff_mappings:
        if ii in player_dict:
            rapm_estimates.append([player_dict[ii], c, (X_df[ii] != 0).sum()])
    rapm_estimates = pd.DataFrame(rapm_estimates, columns = ['Player Name', 'Beta', 'N_stints'])
    return rapm_estimates

### RAPM Computation

The RAPM is computed for every season - while this is not standard, and normally multiple seasons of data are required to stabilize the $\beta$ estimates, we avoid that here due to a few computational bottlenecks.

In [14]:
rapm_estimates = []
for season in season_dfs:
    rapm_estimates.append(compute_RAPM_table(season))

We can take a look at the 2016-17 and 2018-19 leaders and do a little quick mental validation.  Superstars should largely always end up at the top because they are major contributors to their teams.

### 2016-17

In [21]:
rapm_estimates[3].head(20)

,Player Name,Beta,N_stints
0,Stephen Curry,3.030965,1378
1,LeBron James,2.549975,1252
2,Rudy Gobert,2.298129,1418
3,Chris Paul,2.251805,918
4,Patrick Patterson,2.000966,873
5,Kyle Lowry,1.999886,1009
6,Nikola Jokic,1.884757,978
7,JaVale McGee,1.798536,432
8,Jimmy Butler,1.795742,1400
9,Jae Crowder,1.784665,1209


### 2017-18

In [23]:
rapm_estimates[5].head(20)

,Player Name,Beta,N_stints
0,Paul George,3.381254,1405
1,Danny Green,3.345671,952
2,Kevin Durant,3.271545,1341
3,Luol Deng,3.179638,220
4,Jrue Holiday,2.934863,1186
5,Stephen Curry,2.753934,1146
6,Raul Neto,2.748346,293
7,Giannis Antetokounmpo,2.741067,1314
8,Paul Millsap,2.688706,841
9,Andre Drummond,2.481209,1249
